In [ ]:
from utils import back_translate
from utils import compute_bleu_chrf
import json
file_path = "/home/mshahidul/project1/all_tran_data/dataset/Sampled_100_MedlinePlus_eng_spanish_pair.json"
with open(file_path, 'r', encoding='utf-8') as json_file:
    original_file = json.load(json_file)

In [ ]:
info_file_path = "/home/mshahidul/project1/all_tran_data/dataset/medlineplus_info.json"
with open(info_file_path, 'r', encoding='utf-8') as json_file:
    info_data = json.load(json_file)
    # Create a dictionary to map Original_English_sentence to synonyms
sentence_to_synonyms = {item['Original_English_sentence']: item['synonyms'] for item in info_data}

    # Function to find synonyms based on English sentence
def find_synonyms(english_sentence):
    return sentence_to_synonyms.get(english_sentence, "Synonyms not found")

In [ ]:
results_file_path = "/home/mshahidul/project1/results_new/Medline/medlineplus_gpt4_mini_COD_back_translation.json"
with open(results_file_path, 'r', encoding='utf-8') as json_file:
    results_data = json.load(json_file)

# Create a dictionary to map Original_English_sentence to COD_prompt
sentence_to_prompt = {item['Original_English_sentence']: item['COD_prompt'] for item in results_data}

# Function to find COD_prompt based on English sentence
def find_cod_prompt(english_sentence):
    return sentence_to_prompt.get(english_sentence, "Prompt not found")


In [ ]:

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
                    ### Instruction:
                    {}

                    ### Input:
                    {}

                    ### Response:
                    {}"""
max_seq_length=2048
model_name = "unsloth/Qwen2.5-14B-Instruct"
model_name2=model_name.split("/")[1]
from unsloth import FastLanguageModel
model2, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_name, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = None,
        load_in_4bit = False,
    )
FastLanguageModel.for_inference(model2) # Enable native 2x faster inference

In [ ]:
total_score=[]
FastLanguageModel.for_inference(model2) # Enable native 2x faster inference

def inference_without_prompt(ques):
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            f"{find_cod_prompt(ques)}\nUsing the above context translate the input into English to Spanish:", # instruction
            ques, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model2.generate(**inputs, max_new_tokens = 64, use_cache = True)
    ans=tokenizer.batch_decode(outputs)
    start_marker = '### Response:\n'
    end_marker = '<|end_of_text|>'

    # Find the start and end positions
    start_index = ans[0].find(start_marker) + len(start_marker)
    end_index = ans[0].find(end_marker)
    response = ans[0][start_index:end_index].strip()
    try:
        ans=response.split("\n\n")[1]
    except:
        ans=response
    return ans  


def inference_with_prompt(ques):
    # {find_synonyms(ques)}\n
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            f"{find_cod_prompt(ques)}\nUse this context to translate the input into English to Spanish:", # instruction
            ques, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model2.generate(**inputs, max_new_tokens = 64, use_cache = True)
    ans=tokenizer.batch_decode(outputs)
    start_marker = '### Response:\n'
    end_marker = '<|end_of_text|>'

    # Find the start and end positions
    start_index = ans[0].find(start_marker) + len(start_marker)
    end_index = ans[0].find(end_marker)
    response = ans[0][start_index:end_index].strip()
    return response.split("\n\n")[0]



In [ ]:
def inference_with_prompt(ques):
    # {find_synonyms(ques)}\n
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            f"{find_cod_prompt(ques)}\nUse this context to translate the input into English to Spanish:", # instruction
            ques, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model2.generate(**inputs, max_new_tokens = 64, use_cache = True)
    ans=tokenizer.batch_decode(outputs)
    start_marker = '### Response:\n'
    end_marker = '<|end_of_text|>'

    # Find the start and end positions
    start_index = ans[0].find(start_marker) + len(start_marker)
    end_index = ans[0].find(end_marker)
    response = ans[0][start_index:end_index].strip()
    return response.split("\n\n")[0]

In [ ]:
inference_with_prompt("In January 2001, the FDA's Center for Food Safety and Applied Nutrition proposed that developers of bioengineered food submit scientific and safety information to the FDA at least 120 days before the food is marketed.")

In [ ]:
inference_without_prompt("In January 2001, the FDA's Center for Food Safety and Applied Nutrition proposed that developers of bioengineered food submit scientific and safety information to the FDA at least 120 days before the food is marketed.")

## evaluation reference text spanish(real)

In [ ]:

import tqdm
for line in tqdm.tqdm(original_file):
    try:
        spa_tran_prompt = inference_with_prompt(line['english'])
        

        reference_text = [line['spanish']]
        hypothesis_text = spa_tran_prompt
        score1=compute_bleu_chrf(reference_text, hypothesis_text)  

        spa_tran_direct=inference_without_prompt(line['english'])
    
        reference_text = [line['spanish']]
        hypothesis_text = spa_tran_direct
        score2=compute_bleu_chrf(reference_text, hypothesis_text)

        total_score.append({
            "Original_English_sentence": line['english'],
            "Original_Spanish_sentence": line['spanish'],
            "spanish_translation_prompt": spa_tran_prompt,
            "spanish_translation_direct": spa_tran_direct,
            "scores_cod_prompt(bleu and chrf)": score1,
            "scores_direct(bleu and chrf)": score2
        })
    except Exception as e:
        print(f"Error processing line: {line}")
        print(e)

## Evaluation back translation

In [ ]:

# import tqdm
# for line in tqdm.tqdm(original_file):
#     try:
#         spa_tran_prompt = inference_with_prompt(line['english'])
#         back_tran_prompt=back_translate(spa_tran_prompt)

#         reference_text = [line['english']]
#         hypothesis_text = back_tran_prompt
#         score1=compute_bleu_chrf(reference_text, hypothesis_text)  

#         spa_tran_direct=inference_without_prompt(line['english'])
#         back_tran_direct=back_translate(spa_tran_direct)
#         reference_text = [line['english']]
#         hypothesis_text = back_tran_direct
#         score2=compute_bleu_chrf(reference_text, hypothesis_text)

#         total_score.append({
#             "Original_English_sentence": line['english'],
#             "Original_Spanish_sentence": line['spanish'],
#             "spanish_translation_prompt": spa_tran_prompt,
#             "back_translation_prompt": back_tran_prompt,
#             "spanish_translation_direct": spa_tran_direct,
#             "back_translation_direct": back_tran_direct,
#             "scores_cod_prompt(bleu and chrf)": score1,
#             "scores_direct(bleu and chrf)": score2
#         })
#     except Exception as e:
#         print(f"Error processing line: {line}")
#         print(e)

In [ ]:
output_file_path = f"/home/mshahidul/project1/results_new/medline_{model_name2}_direct_and_COD_translation.json"
with open(output_file_path, 'w', encoding='utf-8') as json_file: 
    json.dump(total_score, json_file, ensure_ascii=False, indent=4)


In [ ]:
# Initialize variables to store the sum of scores
total_bleu_cod_prompt = 0

total_bleu_direct = 0


# Iterate through the total_score list to sum up the scores
for score in total_score:
    total_bleu_cod_prompt += score["scores_cod_prompt(bleu and chrf)"]['bleu_score']
    
    total_bleu_direct += score["scores_direct(bleu and chrf)"]['bleu_score']


# Calculate the average scores
num_entries = len(total_score)
avg_bleu_cod_prompt = total_bleu_cod_prompt / num_entries

avg_bleu_direct = total_bleu_direct / num_entries

print(model_name)
# Print the average scores
print(f"{model_name2} (COD prompt): {avg_bleu_cod_prompt}")

print(f"{model_name2} (Direct): {avg_bleu_direct}")
